In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install tensorflow

In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
# Multi-Dataset ALS pipeline: tuning, NAS, evaluation, meta-learner, save artifacts
import os, time
import numpy as np
import pandas as pd

from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, roc_auc_score,
    f1_score, precision_score, recall_score, confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# Optional: imblearn for SMOTE inside pipelines
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
    HAS_IMBLEARN = True
except Exception:
    HAS_IMBLEARN = False

# ---------------------------
# Configuration
# ---------------------------
DATASET_CONFIGS = {
    "dataset_1": {
        "path": "/content/drive/MyDrive/Neurodivergent/Minsk2020_ALS_dataset.csv",
        "name": "Dataset_1"
    },
    "dataset_2": {
        "path": "/content/drive/MyDrive/Neurodivergent/Parkinsson disease.csv",
        "name": "Dataset_2"
    },
    "dataset_3": {
        "path": "/content/drive/MyDrive/Neurodivergent/alzheimers_disease_data.csv",
        "name": "Dataset_3"
    },
    "dataset_4": {
        "path": "/content/drive/MyDrive/Neurodivergent/dataset_4.csv",
        "name": "Dataset_4"
    }
}

RANDOM_STATE = 42
BASE_RESULTS_DIR = "/content/multi_dataset_results"
os.makedirs(BASE_RESULTS_DIR, exist_ok=True)

# ---------------------------
# Helper: OneHotEncoder version-safe
# ---------------------------
def get_onehot_encoder(**kwargs):
    try:
        # sklearn >= 1.2
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False, **kwargs)
    except TypeError:
        # older sklearn
        return OneHotEncoder(handle_unknown="ignore", sparse=False, **kwargs)

# ---------------------------
# Global NAS Architecture Storage
# ---------------------------
SHARED_NAS_ARCHITECTURE = None

# ---------------------------
# Expanded NAS using keras-tuner (with architecture sharing)
# ---------------------------
def run_keras_nas_search(X_train, y_train, max_trials=6, epochs=8, batch_size=32, verbose=0):
    """
    Use keras-tuner to search MLP architectures on the first dataset.
    Returns the best keras.Model instance and architecture config.
    """
    try:
        import tensorflow as tf
        from tensorflow import keras
    except Exception:
        print("TensorFlow not available — skipping NAS.")
        return None, None

    # try import keras_tuner
    kt = None
    try:
        import keras_tuner as kt
    except Exception:
        try:
            import kerastuner as kt
        except Exception:
            kt = None

    if kt is None:
        print("Keras Tuner not available — skipping NAS.")
        return None, None

    input_shape = int(X_train.shape[1])
    n_classes = len(np.unique(y_train))

    def build_model(hp):
        model = keras.Sequential()
        num_layers = hp.Int("num_layers", 1, 4)
        activation = hp.Choice("activation", ["relu", "tanh"])
        learning_rate = hp.Choice("lr", [1e-2, 1e-3, 1e-4])

        # Store architecture config for reuse
        arch_config = {
            "num_layers": num_layers,
            "activation": activation,
            "learning_rate": learning_rate,
            "layers": []
        }

        for i in range(num_layers):
            units = hp.Int(f"units_{i}", 32, 256, step=32)
            use_batchnorm = hp.Boolean(f"batchnorm_{i}")
            use_dropout = hp.Boolean(f"dropout_{i}")
            dropout_rate = hp.Float(f"dropout_rate_{i}", 0.1, 0.5, step=0.1) if use_dropout else 0.0

            arch_config["layers"].append({
                "units": units,
                "use_batchnorm": use_batchnorm,
                "use_dropout": use_dropout,
                "dropout_rate": dropout_rate
            })

            if i == 0:
                model.add(keras.layers.Dense(units, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(keras.layers.Dense(units, activation=activation))

            if use_batchnorm:
                model.add(keras.layers.BatchNormalization())
            if use_dropout:
                model.add(keras.layers.Dropout(rate=dropout_rate))

        # output layer
        if n_classes > 2:
            model.add(keras.layers.Dense(n_classes, activation="softmax"))
            loss = "sparse_categorical_crossentropy"
        else:
            model.add(keras.layers.Dense(1, activation="sigmoid"))
            loss = "binary_crossentropy"

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss=loss,
            metrics=["accuracy"]
        )

        # Store config in model for later extraction
        model._arch_config = arch_config
        return model

    tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=max_trials, overwrite=True,
                            directory="nas_search", project_name="multi_dataset_nas")
    tuner.search(X_train, y_train, validation_split=0.15, epochs=epochs, batch_size=batch_size, verbose=verbose)
    best_models = tuner.get_best_models(num_models=1)

    if best_models:
        best_model = best_models[0]
        # Extract architecture config
        arch_config = getattr(best_model, '_arch_config', None)
        return best_model, arch_config
    return None, None

def build_model_from_architecture(arch_config, input_shape, n_classes):
    """
    Build a Keras model using the shared architecture configuration.
    """
    try:
        import tensorflow as tf
        from tensorflow import keras
    except Exception:
        return None

    if arch_config is None:
        return None

    model = keras.Sequential()

    for i, layer_config in enumerate(arch_config["layers"]):
        if i == 0:
            model.add(keras.layers.Dense(
                layer_config["units"],
                activation=arch_config["activation"],
                input_shape=(input_shape,)
            ))
        else:
            model.add(keras.layers.Dense(
                layer_config["units"],
                activation=arch_config["activation"]
            ))

        if layer_config["use_batchnorm"]:
            model.add(keras.layers.BatchNormalization())
        if layer_config["use_dropout"]:
            model.add(keras.layers.Dropout(rate=layer_config["dropout_rate"]))

    # output layer
    if n_classes > 2:
        model.add(keras.layers.Dense(n_classes, activation="softmax"))
        loss = "sparse_categorical_crossentropy"
    else:
        model.add(keras.layers.Dense(1, activation="sigmoid"))
        loss = "binary_crossentropy"

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=arch_config["learning_rate"]),
        loss=loss,
        metrics=["accuracy"]
    )

    return model

# ---------------------------
# Keras wrapper
# ---------------------------
class KerasWrapper:
    def __init__(self, model, epochs=10, batch_size=32, verbose=0):
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self._is_fitted = False

    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        self._is_fitted = True
        return self

    def predict(self, X):
        proba = self.model.predict(X, batch_size=self.batch_size, verbose=0)
        proba = np.asarray(proba)
        if proba.ndim == 1 or proba.shape[1] == 1:
            # binary
            p = proba.ravel()
            return (p > 0.5).astype(int)
        else:
            return np.argmax(proba, axis=1)

    def predict_proba(self, X):
        proba = self.model.predict(X, batch_size=self.batch_size, verbose=0)
        proba = np.asarray(proba)
        if proba.ndim == 1 or proba.shape[1] == 1:
            p = proba.ravel()
            return np.vstack([1 - p, p]).T
        else:
            return proba

# ---------------------------
# Dataset processing functions
# ---------------------------
def detect_target(df):
    kws = ["diagnosis", "label", "class", "target", "outcome", "result", "status"]
    for c in df.columns:
        if any(k in c.lower() for k in kws):
            return c
    return df.columns[-1]

def load_and_preprocess_dataset(dataset_path, dataset_name):
    """Load and preprocess a single dataset."""
    print(f"\n{'='*50}")
    print(f"Processing {dataset_name}")
    print(f"{'='*50}")

    # Load dataset - handle both CSV and Excel files
    if dataset_path.lower().endswith(('.xlsx', '.xls')):
        df = pd.read_excel(dataset_path)
        print(f"Loaded {dataset_name} (Excel):", df.shape)
    else:
        df = pd.read_csv(dataset_path)
        print(f"Loaded {dataset_name} (CSV):", df.shape)

    # Drop id-like columns
    drop_like = [c for c in df.columns if c.lower() in ["id", "name", "subjectid", "patientid", "caseid"]]
    df = df.drop(columns=drop_like, errors="ignore")

    # Detect target column
    target_col = detect_target(df)
    print(f"Target detected for {dataset_name}:", target_col)

    # Ensure numeric labels
    if df[target_col].dtype == "object":
        le = LabelEncoder()
        df[target_col] = le.fit_transform(df[target_col].astype(str))
    elif not np.issubdtype(df[target_col].dtype, np.number):
        df[target_col] = pd.factorize(df[target_col])[0]

    X = df.drop(columns=[target_col])
    y = df[target_col].values

    return X, y, target_col

# ---------------------------
# Model definitions
# ---------------------------
def get_models_and_spaces():
    """Get candidate models and their parameter spaces."""
    return {
        "rf": (
            RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
            {"clf__n_estimators": [100, 200, 400], "clf__max_depth": [None, 6, 12, 20], "clf__min_samples_split": [2, 5, 10]}
        ),
        "xgb": (
            XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=RANDOM_STATE, n_jobs=1),
            {"clf__n_estimators": [100, 200, 400], "clf__max_depth": [3, 6, 10], "clf__learning_rate": [0.01, 0.05, 0.1]}
        ),
        "svc": (
            SVC(probability=True, random_state=RANDOM_STATE),
            {"clf__C": [0.1, 1, 10], "clf__gamma": ["scale", "auto"]}
        ),
        "knn": (
            KNeighborsClassifier(),
            {"clf__n_neighbors": [3, 5, 7, 9], "clf__weights": ["uniform", "distance"]}
        ),
        "lr": (
            LogisticRegression(max_iter=2000),
            {"clf__C": [0.01, 0.1, 1, 10]}
        ),
        "nb": (
            GaussianNB(),
            {"clf__var_smoothing": [1e-9, 1e-8, 1e-7]}
        )
    }

# ---------------------------
# Training and evaluation functions
# ---------------------------
def create_preprocessing_pipeline(X_df):
    """Create preprocessing pipeline for a dataset."""
    num_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_df.columns if c not in num_cols]

    print(f"Numeric cols: {len(num_cols)}, Categorical cols: {len(cat_cols)}")

    numeric_transformer = Pipeline([
        ("imputer", IterativeImputer(random_state=RANDOM_STATE, max_iter=10)),
        ("scaler", RobustScaler())
    ])
    categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", get_onehot_encoder())
    ])

    preprocessor = ColumnTransformer([
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ], remainder="drop", sparse_threshold=0)

    return preprocessor

def tune_and_evaluate(model_name, estimator, param_dist, X_df, y_arr, preprocessor, n_iter=12, cv_splits=3):
    """Tune and evaluate a single model."""
    use_pipeline_cls = ImbPipeline if HAS_IMBLEARN else Pipeline

    # Build pipeline
    pipe_steps = [("pre", preprocessor)]
    if HAS_IMBLEARN:
        from collections import Counter
        counts = Counter(y_arr)
        if len(counts) > 1:
            imbalance = max(counts.values()) / min(counts.values())
        else:
            imbalance = 1.0
        if imbalance > 1.5:
            pipe_steps.append(("smote", SMOTE(random_state=RANDOM_STATE)))

    k_default = min(50, max(5, int(max(5, X_df.shape[1] * 0.5))))
    pipe_steps.append(("select", SelectKBest(score_func=f_classif, k=k_default)))
    pipe_steps.append(("clf", estimator))
    pipeline = use_pipeline_cls(steps=pipe_steps)

    rskf = RepeatedStratifiedKFold(n_splits=cv_splits, n_repeats=2, random_state=RANDOM_STATE)
    search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=n_iter,
                                cv=rskf, scoring="balanced_accuracy", n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
    start = time.time()
    search.fit(X_df, y_arr)
    elapsed = time.time() - start
    best = search.best_estimator_
    print(f"[{model_name}] best score: {search.best_score_:.4f} time: {elapsed:.1f}s")
    return best, search

def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    """Evaluate a single model."""
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = None
    try:
        y_proba = model.predict_proba(X_test)
    except Exception:
        y_proba = None

    metrics = {
        "model": name,
        "accuracy": accuracy_score(y_test, y_pred),
        "balanced_accuracy": balanced_accuracy_score(y_test, y_pred),
        "f1_weighted": f1_score(y_test, y_pred, average="weighted"),
        "precision_weighted": precision_score(y_test, y_pred, average="weighted", zero_division=0),
        "recall_weighted": recall_score(y_test, y_pred, average="weighted", zero_division=0),
    }

    if y_proba is not None:
        try:
            if len(np.unique(y_test)) == 2:
                metrics["roc_auc"] = roc_auc_score(y_test, y_proba[:, 1])
            else:
                metrics["roc_auc"] = roc_auc_score(y_test, y_proba, multi_class="ovr")
        except Exception:
            metrics["roc_auc"] = None
    else:
        metrics["roc_auc"] = None

    print(f"\n=== {name.upper()} ===")
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification report:\n", classification_report(y_test, y_pred))
    return metrics

def process_single_dataset(dataset_key, dataset_config, is_first_dataset=False):
    """Process a single dataset with the NAS architecture."""
    global SHARED_NAS_ARCHITECTURE

    dataset_name = dataset_config["name"]
    dataset_path = dataset_config["path"]

    # Create results directory for this dataset
    results_dir = os.path.join(BASE_RESULTS_DIR, dataset_key)
    os.makedirs(results_dir, exist_ok=True)

    # Load and preprocess dataset
    X, y, target_col = load_and_preprocess_dataset(dataset_path, dataset_name)

    # Train/holdout split
    X_train_df, X_hold_df, y_train, y_hold = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE,
        stratify=y if len(np.unique(y)) > 1 else None
    )

    # Create preprocessing pipeline
    preprocessor = create_preprocessing_pipeline(X_train_df)

    # Get models and parameter spaces
    models_and_spaces = get_models_and_spaces()

    # Tune traditional ML models
    print(f"\nTuning traditional ML models for {dataset_name}...")
    tuned_models = {}
    for name, (est, space) in models_and_spaces.items():
        try:
            best_est, search_obj = tune_and_evaluate(name, est, space, X_train_df, y_train, preprocessor)
            tuned_models[name] = best_est
        except Exception as e:
            print(f"Failed tuning {name}: {e}")

    # Handle NAS model
    nas_model = None
    if is_first_dataset:
        # First dataset: perform NAS search
        print(f"\nPerforming NAS search on {dataset_name}...")
        try:
            X_train_pre = preprocessor.fit_transform(X_train_df)
            X_train_pre = np.asarray(X_train_pre, dtype=np.float32)
            print(f"Preprocessed training data shape: {X_train_pre.shape}")
            print(f"Number of classes: {len(np.unique(y_train))}")

            nas_model, SHARED_NAS_ARCHITECTURE = run_keras_nas_search(
                X_train_pre, y_train, max_trials=6, epochs=8, batch_size=32, verbose=0
            )
            if nas_model is not None and SHARED_NAS_ARCHITECTURE is not None:
                print(f"NAS discovered architecture for {dataset_name}")
                print("Architecture details:", SHARED_NAS_ARCHITECTURE)
                nas_model.summary()
            else:
                print("NAS failed or not available - checking individual components:")
                print(f"  nas_model is None: {nas_model is None}")
                print(f"  SHARED_NAS_ARCHITECTURE is None: {SHARED_NAS_ARCHITECTURE is None}")
        except Exception as e:
            print(f"NAS attempt failed: {e}")
            import traceback
            traceback.print_exc()
    else:
        # Subsequent datasets: use shared architecture
        if SHARED_NAS_ARCHITECTURE is not None:
            print(f"\nApplying shared NAS architecture to {dataset_name}...")
            print("Using architecture:", SHARED_NAS_ARCHITECTURE)
            try:
                X_train_pre = preprocessor.fit_transform(X_train_df)
                X_train_pre = np.asarray(X_train_pre, dtype=np.float32)
                input_shape = X_train_pre.shape[1]
                n_classes = len(np.unique(y_train))

                print(f"Building NAS model with input_shape={input_shape}, n_classes={n_classes}")
                nas_model = build_model_from_architecture(SHARED_NAS_ARCHITECTURE, input_shape, n_classes)
                if nas_model is not None:
                    print(f"Successfully built NAS model for {dataset_name}")
                    nas_model.summary()
                else:
                    print("Failed to build NAS model from shared architecture")
            except Exception as e:
                print(f"Failed to apply shared NAS architecture: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("No shared NAS architecture available")

    # Add NAS model to tuned_models if available
    if nas_model is not None:
        keras_wrapper = KerasWrapper(nas_model, epochs=8, batch_size=32, verbose=0)
        nas_pipeline = Pipeline([("pre", preprocessor), ("keras", keras_wrapper)])
        tuned_models["nas"] = nas_pipeline

    # Evaluate all models
    print(f"\nEvaluating all models for {dataset_name}...")
    all_results = []
    for name, model in tuned_models.items():
        try:
            metrics = evaluate_model(name, model, X_train_df, y_train, X_hold_df, y_hold)
            all_results.append(metrics)
        except Exception as e:
            print(f"Evaluation failed for {name}: {e}")

    # Create results summary
    if all_results:
        results_df = pd.DataFrame(all_results).sort_values(by="balanced_accuracy", ascending=False).reset_index(drop=True)
        print(f"\nSummary of all models for {dataset_name}:")
        print(results_df)

        # Save results
        results_df.to_csv(os.path.join(results_dir, f"{dataset_key}_results.csv"), index=False)

        # Get best model
        best_row = results_df.iloc[0]
        best_model_name = best_row["model"]
        print(f"\n🔥 Best model for {dataset_name} is: {best_model_name}")
        print(f"Best balanced accuracy: {best_row['balanced_accuracy']:.4f}")

        # Save best model
        import joblib
        try:
            best_pipeline = tuned_models[best_model_name]
            if best_model_name == "nas" and nas_model is not None:
                # Save Keras model separately
                keras_save_path = os.path.join(results_dir, f"{dataset_key}_nas_best_model.keras")
                nas_model.save(keras_save_path)
                joblib.dump({"preprocessor": preprocessor, "note": "NAS Keras model saved separately."},
                           os.path.join(results_dir, f"{dataset_key}_preprocessor_for_nas.joblib"))
                print(f"Saved NAS Keras model to: {keras_save_path}")
            else:
                save_path = os.path.join(results_dir, f"{dataset_key}_best_{best_model_name}.joblib")
                joblib.dump(best_pipeline, save_path)
                print(f"Saved best sklearn pipeline to: {save_path}")
        except Exception as e:
            print(f"Saving best model failed: {e}")

        return {
            "dataset_name": dataset_name,
            "results_df": results_df,
            "best_model_name": best_model_name,
            "best_metrics": best_row,
            "tuned_models": tuned_models,
            "preprocessor": preprocessor
        }
    else:
        print(f"No models evaluated successfully for {dataset_name}")
        return None

# ---------------------------
# Main execution
# ---------------------------
def main():
    """Main function to process all datasets."""
    print("Starting Multi-Dataset ALS Pipeline with NAS")
    print(f"Processing {len(DATASET_CONFIGS)} datasets")

    all_dataset_results = {}

    # Process each dataset
    for i, (dataset_key, dataset_config) in enumerate(DATASET_CONFIGS.items()):
        is_first = (i == 0)  # First dataset performs NAS search

        try:
            result = process_single_dataset(dataset_key, dataset_config, is_first_dataset=is_first)
            if result:
                all_dataset_results[dataset_key] = result
        except Exception as e:
            print(f"Failed to process {dataset_key}: {e}")
            continue

    # Cross-dataset analysis
    if all_dataset_results:
        print(f"\n{'='*60}")
        print("CROSS-DATASET ANALYSIS")
        print(f"{'='*60}")

        # Compare best models across datasets
        comparison_data = []
        for dataset_key, result in all_dataset_results.items():
            comparison_data.append({
                "Dataset": result["dataset_name"],
                "Best_Model": result["best_model_name"],
                "Balanced_Accuracy": result["best_metrics"]["balanced_accuracy"],
                "F1_Weighted": result["best_metrics"]["f1_weighted"],
                "ROC_AUC": result["best_metrics"].get("roc_auc", "N/A")
            })

        comparison_df = pd.DataFrame(comparison_data)
        print("\nCross-dataset comparison:")
        print(comparison_df)

        # Save cross-dataset comparison
        comparison_df.to_csv(os.path.join(BASE_RESULTS_DIR, "cross_dataset_comparison.csv"), index=False)

        # Save shared NAS architecture if available
        if SHARED_NAS_ARCHITECTURE is not None:
            import joblib
            try:
                joblib.dump(SHARED_NAS_ARCHITECTURE, os.path.join(BASE_RESULTS_DIR, "shared_nas_architecture.joblib"))
                print(f"\nSaved shared NAS architecture to: {BASE_RESULTS_DIR}/shared_nas_architecture.joblib")
            except Exception as e:
                print(f"Failed to save shared NAS architecture: {e}")

        # Model performance analysis
        model_performance = {}
        for dataset_key, result in all_dataset_results.items():
            for _, row in result["results_df"].iterrows():
                model_name = row["model"]
                if model_name not in model_performance:
                    model_performance[model_name] = []
                model_performance[model_name].append(row["balanced_accuracy"])

        print("\nAverage model performance across datasets:")
        for model, scores in model_performance.items():
            avg_score = np.mean(scores)
            std_score = np.std(scores)
            print(f"{model}: {avg_score:.4f} ± {std_score:.4f}")

    print(f"\nPipeline completed. Results saved in: {BASE_RESULTS_DIR}")

if __name__ == "__main__":
    main()

Starting Multi-Dataset ALS Pipeline with NAS
Processing 4 datasets

Processing Dataset_1
Loaded Dataset_1 (CSV): (64, 135)
Target detected for Dataset_1: Diagnosis (ALS)
Numeric cols: 132, Categorical cols: 1

Tuning traditional ML models for Dataset_1...
[rf] best score: 0.7569 time: 79.7s


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [13:01:52] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=12. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[xgb] best score: 0.7095 time: 41.2s
[svc] best score: 0.7118 time: 19.8s


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 8 is smaller than n_iter=12. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[knn] best score: 0.6956 time: 26.5s


...


=== LR ===
Confusion matrix:
 [[220  58]
 [ 23 129]]
Classification report:
               precision    recall  f1-score   support

           0       0.91      0.79      0.84       278
           1       0.69      0.85      0.76       152

    accuracy                           0.81       430
   macro avg       0.80      0.82      0.80       430
weighted avg       0.83      0.81      0.82       430



/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [32] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



=== NB ===
Confusion matrix:
 [[215  63]
 [ 30 122]]
Classification report:
               precision    recall  f1-score   support

           0       0.88      0.77      0.82       278
           1       0.66      0.80      0.72       152

    accuracy                           0.78       430
   macro avg       0.77      0.79      0.77       430
weighted avg       0.80      0.78      0.79       430



/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(



=== NAS ===
Confusion matrix:
 [[214  64]
 [ 23 129]]
Classification report:
               precision    recall  f1-score   support

           0       0.90      0.77      0.83       278
           1       0.67      0.85      0.75       152

    accuracy                           0.80       430
   macro avg       0.79      0.81      0.79       430
weighted avg       0.82      0.80      0.80       430


Summary of all models for Dataset_3:
  model  accuracy  balanced_accuracy  f1_weighted  precision_weighted  \
0    rf  0.951163           0.942848     0.950972            0.951064   
1   xgb  0.946512           0.939251     0.946389            0.946366   
2   svc  0.855814           0.846744     0.856423            0.857380   
3    lr  0.811628           0.820026     0.815025            0.829169   
4   nas  0.797674           0.809234     0.801643            0.820039   
5    nb  0.783721           0.788006     0.787487            0.800458   
6   knn  0.781395           0.784717     0.78

In [ ]:
# ============================================
# Unified ML pipeline for multiple neuro datasets with Meta-Model
# - 6 base models + Stacking Meta-Model
# - Robust target detection
# - Full evaluation metrics
# ============================================

import re, difflib, warnings, time
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, roc_auc_score,
    f1_score, precision_score, recall_score, confusion_matrix, classification_report
)

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# ---------------------------
# Dataset Paths + Target Hints
# ---------------------------
datasets = {
    "Autism": ("/content/drive/MyDrive/Neurodivergent/autism_screening.csv", "Class/ASD"),
    "Alzheimer": ("/content/drive/MyDrive/Neurodivergent/alzheimers_disease_data.csv", "Class"),
    "ALS": ("/content/drive/MyDrive/Neurodivergent/Minsk2020_ALS_dataset.csv", "diagnosis"),
    "Parkinson": ("/content/drive/MyDrive/Neurodivergent/Parkinsson disease.csv", "status"),
}

# ---------------------------
# Target detection helper
# ---------------------------
def guess_target(df: pd.DataFrame, preferred: str | None = None):
    cols = list(df.columns)
    norm = lambda s: re.sub(r'[^a-z0-9]+', '', str(s).lower())
    if preferred and preferred in cols:
        return preferred
    if preferred:
        norm_pref = norm(preferred)
        for c in cols:
            if norm(c) == norm_pref:
                return c
        close = difflib.get_close_matches(preferred, cols, n=1, cutoff=0.6)
        if close:
            return close[0]
    kw = ['class', 'asd', 'diagnosis', 'label', 'target', 'status', 'group', 'outcome', 'response', 'disease', 'result']
    for c in cols:
        if any(k in norm(c) for k in kw):
            return c
    return cols[-1]

# ---------------------------
# Extended Evaluation helper
# ---------------------------
def safe_roc_auc(y_true, model, X_test):
    """Compute ROC AUC if possible (binary or multiclass). Returns 'NA' if not applicable."""
    try:
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(X_test)
        elif hasattr(model, "decision_function"):
            dfun = model.decision_function(X_test)
            if dfun.ndim == 1:
                proba = np.vstack([1 - 1/(1+np.exp(dfun)), 1/(1+np.exp(-dfun))]).T
            else:
                e = np.exp(dfun - np.max(dfun, axis=1, keepdims=True))
                proba = e / np.sum(e, axis=1, keepdims=True)
        else:
            return "NA"

        classes = np.unique(y_true)
        if len(classes) == 2:
            return roc_auc_score(y_true, proba[:, 1])
        elif len(classes) > 2:
            return roc_auc_score(y_true, proba, multi_class="ovr")
        return "NA"
    except Exception:
        return "NA"

def evaluate_classification(models, X_train, X_test, y_train, y_test):
    results = []
    reports = {}
    conf_mats = {}

    for name, model in models.items():
        try:
            start = time.time()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            elapsed = round(time.time() - start, 3)

            roc = safe_roc_auc(y_test, model, X_test)
            acc = accuracy_score(y_test, y_pred)
            bal_acc = balanced_accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average="weighted")
            prec = precision_score(y_test, y_pred, average="weighted", zero_division=0)
            rec = recall_score(y_test, y_pred, average="weighted", zero_division=0)

            results.append([name, acc, bal_acc, roc, f1, prec, rec, elapsed])
            conf_mats[name] = confusion_matrix(y_test, y_pred)
            reports[name] = classification_report(y_test, y_pred, output_dict=True)
        except Exception as e:
            results.append([name, "ERR", "ERR", "ERR", "ERR", "ERR", "ERR", f"failed: {e}"])

    df_results = pd.DataFrame(
        results,
        columns=["Model","Accuracy","Balanced Accuracy","ROC AUC","F1 Score","Precision","Recall","Time Taken"]
    )
    return df_results, reports, conf_mats

# ---------------------------
# Base + Stacking Meta-Model
# ---------------------------
def get_models():
    base_models = {
        "Logistic Regression": LogisticRegression(max_iter=2000),
        "Random Forest": RandomForestClassifier(),
        "SVM (RBF)": SVC(probability=True),
        "KNN": KNeighborsClassifier(),
        "Naive Bayes": GaussianNB(),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    }

    # Define stacking meta-model
    stack_estimators = [
        ("lr", LogisticRegression(max_iter=2000)),
        ("rf", RandomForestClassifier()),
        ("svm", SVC(probability=True)),
        ("knn", KNeighborsClassifier()),
        ("nb", GaussianNB()),
        ("xgb", XGBClassifier(use_label_encoder=False, eval_metric="logloss"))
    ]
    meta_model = LogisticRegression(max_iter=2000)
    stacking = StackingClassifier(
        estimators=stack_estimators,
        final_estimator=meta_model,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )
    base_models["Stacking (Meta-Model)"] = stacking
    return base_models

# ---------------------------
# Main Loop
# ---------------------------
models = get_models()
autism_train_df = None
autism_test_df  = None

for ds_name, (path, target_hint) in datasets.items():
    print(f"\n============================\n📌 Results for {ds_name} Dataset\n============================")
    df = pd.read_csv(path)

    # Save autism train/test separately
    if ds_name == "Autism_Train":
        autism_train_df = df.copy()
    if ds_name == "Autism_Test":
        autism_test_df = df.copy()
        print("  Test dataset (unlabeled). Skipping evaluation.")
        continue

    # Drop ID-like columns
    drop_like = [c for c in df.columns if c.lower() in ["id","name","subjectid","patientid","caseid"]]
    df = df.drop(columns=drop_like, errors="ignore")

    # Auto-detect target
    target = guess_target(df, preferred=target_hint)
    print(f"  Target column selected: '{target}' (hint was '{target_hint}')")

    # Encode target if categorical
    if df[target].dtype == 'object':
        le = LabelEncoder()
        df[target] = le.fit_transform(df[target].astype(str))

    # Features / target
    X = df.drop(columns=[target], errors="ignore")
    y = df[target]

    # Handle missing
    num_cols = X.select_dtypes(include=[np.number]).columns
    cat_cols = [c for c in X.columns if c not in num_cols]
    if len(num_cols) > 0:
        X[num_cols] = X[num_cols].fillna(X[num_cols].median())
    for c in cat_cols:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].mode().iloc[0])
    X = pd.get_dummies(X, drop_first=True)

    # Train/validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y if y.nunique() > 1 else None
    )

    # Scale
    scaler = StandardScaler(with_mean=False)
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)

    # Evaluate
    results, reports, conf_mats = evaluate_classification(models, X_train, X_val, y_train, y_val)
    display(results)

# ---------------------------
# Autism: Train full & predict test with best model
# ---------------------------
if autism_train_df is not None and autism_test_df is not None:
    print("\n============================\n📌 Autism: Final Model & Test Predictions\n============================")

    drop_like = [c for c in autism_train_df.columns if c.lower() in ["id","name","subjectid","patientid","caseid"]]
    autism_train_df = autism_train_df.drop(columns=drop_like, errors="ignore")
    autism_test_df  = autism_test_df.drop(columns=drop_like, errors="ignore")

    target = guess_target(autism_train_df, preferred="Class/ASD")
    if autism_train_df[target].dtype == 'object':
        le = LabelEncoder()
        autism_train_df[target] = le.fit_transform(autism_train_df[target].astype(str))

    X = autism_train_df.drop(columns=[target])
    y = autism_train_df[target]

    # Missing values
    num_cols = X.select_dtypes(include=[np.number]).columns
    cat_cols = [c for c in X.columns if c not in num_cols]
    if len(num_cols) > 0:
        X[num_cols] = X[num_cols].fillna(X[num_cols].median())
    for c in cat_cols:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].mode().iloc[0])
    X = pd.get_dummies(X, drop_first=True)

    X_test_final = autism_test_df.copy()
    for c in cat_cols:
        if c in X_test_final.columns and X_test_final[c].isna().any():
            X_test_final[c] = X_test_final[c].fillna(X_test_final[c].mode().iloc[0])
    X_test_final = pd.get_dummies(X_test_final, drop_first=True)
    X_test_final = X_test_final.reindex(columns=X.columns, fill_value=0)

    # Scale
    scaler = StandardScaler(with_mean=False)
    X_scaled = scaler.fit_transform(X)
    X_test_scaled = scaler.transform(X_test_final)

    # Pick best model: use Stacking meta-model as candidate
    best_model = StackingClassifier(
        estimators=[
            ("lr", LogisticRegression(max_iter=2000)),
            ("rf", RandomForestClassifier()),
            ("svm", SVC(probability=True)),
            ("knn", KNeighborsClassifier()),
            ("nb", GaussianNB()),
            ("xgb", XGBClassifier(use_label_encoder=False, eval_metric="logloss"))
        ],
        final_estimator=LogisticRegression(max_iter=2000),
        cv=5,
        n_jobs=-1,
        passthrough=False
    )
    best_model.fit(X_scaled, y)
    test_preds = best_model.predict(X_test_scaled)

    # Save predictions
    submission = pd.DataFrame({
        "ID": range(1, len(test_preds)+1),
        "Predicted_Class/ASD": test_preds
    })
    submission.to_csv("/content/test_predictions.csv", index=False)
    print("\n Autism test predictions saved as '/content/test_predictions.csv'")



📌 Results for Autism Dataset
  Target column selected: 'Class/ASD' (hint was 'Class/ASD')


,Model,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Precision,Recall,Time Taken
0,Logistic Regression,0.964539,0.942514,0.997701,0.964065,0.964734,0.964539,0.013
1,Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.219
2,SVM (RBF),0.907801,0.845554,0.981094,0.903539,0.909668,0.907801,0.222
3,KNN,0.836879,0.788707,0.919775,0.836186,0.835589,0.836879,0.006
4,Naive Bayes,0.312057,0.512519,0.512519,0.218251,0.658459,0.312057,0.003
5,XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.038
6,Stacking (Meta-Model),1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.561



📌 Results for Alzheimer Dataset
  Target column selected: 'CardiovascularDisease' (hint was 'Class')


,Model,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Precision,Recall,Time Taken
0,Logistic Regression,0.855814,0.500000,0.442234,0.789322,0.732418,0.855814,0.133
1,Random Forest,0.855814,0.500000,0.477932,0.789322,0.732418,0.855814,1.072
2,SVM (RBF),0.855814,0.500000,0.489437,0.789322,0.732418,0.855814,1.096
3,KNN,0.841860,0.498554,0.501315,0.786269,0.750130,0.841860,0.021
4,Naive Bayes,0.853488,0.498641,0.455645,0.788165,0.732130,0.853488,0.006
5,XGBoost,0.844186,0.493207,0.455075,0.783507,0.730966,0.844186,0.460
6,Stacking (Meta-Model),0.855814,0.500000,0.484967,0.789322,0.732418,0.855814,6.942



📌 Results for ALS Dataset
  Target column selected: 'Diagnosis (ALS)' (hint was 'diagnosis')


...

from matplotlib import pyplot as plt
results['F1 Score'].plot(kind='line', figsize=(8, 4), title='F1 Score')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results, x='Accuracy', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results, x='Balanced Accuracy', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results, x='ROC AUC', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results, x='F1 Score', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# ============================================
# Full AutoML Pipeline for Multiple Neuro Datasets
# ============================================

import re, difflib, warnings, time
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE

import joblib

# ============================================
# Helper Functions
# ============================================

def guess_target(df, preferred=None):
    cols = list(df.columns)
    norm = lambda s: re.sub(r'[^a-z0-9]+', '', str(s).lower())
    if preferred and preferred in cols:
        return preferred
    if preferred:
        norm_pref = norm(preferred)
        for c in cols:
            if norm(c) == norm_pref:
                return c
        close = difflib.get_close_matches(preferred, cols, n=1, cutoff=0.6)
        if close:
            return close[0]
    kw = ['class', 'asd', 'diagnosis', 'label', 'target', 'status', 'group', 'outcome', 'response', 'disease', 'result']
    for c in cols:
        if any(k in norm(c) for k in kw):
            return c
    return cols[-1]

def safe_roc_auc(y_true, model, X_test):
    try:
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(X_test)
        elif hasattr(model, "decision_function"):
            dfun = model.decision_function(X_test)
            if dfun.ndim == 1:
                proba = np.vstack([1 - 1/(1+np.exp(dfun)), 1/(1+np.exp(-dfun))]).T
            else:
                e = np.exp(dfun - np.max(dfun, axis=1, keepdims=True))
                proba = e / np.sum(e, axis=1, keepdims=True)
        else:
            return "NA"
        classes = np.unique(y_true)
        if len(classes) == 2:
            return roc_auc_score(y_true, proba[:, 1])
        elif len(classes) > 2:
            return roc_auc_score(y_true, proba, multi_class="ovr")
        return "NA"
    except:
        return "NA"

# ============================================
# AutoML Class for Multiple Datasets
# ============================================

class AutoML:
    def __init__(self, datasets, n_features=20, balance=True, hyperparam_tuning=False):
        """
        datasets: dict with {dataset_name: (csv_path, target_hint)}
        """
        self.datasets = datasets
        self.n_features = n_features
        self.balance = balance
        self.hyperparam_tuning = hyperparam_tuning
        self.scalers = {}
        self.best_models = {}
        self.results = {}

    # ---------------------------
    # Base + Stacking models
    # ---------------------------
    def get_models(self):
        base_models = [
            ('lr', LogisticRegression(max_iter=2000)),
            ('rf', RandomForestClassifier()),
            ('svm', SVC(probability=True)),
            ('knn', KNeighborsClassifier()),
            ('nb', GaussianNB()),
            ('xgb', XGBClassifier(use_label_encoder=False, eval_metric="logloss"))
        ]
        stacking = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(max_iter=2000), cv=5, n_jobs=-1)
        return {
            "Logistic Regression": LogisticRegression(max_iter=2000),
            "Random Forest": RandomForestClassifier(),
            "SVM": SVC(probability=True),
            "KNN": KNeighborsClassifier(),
            "Naive Bayes": GaussianNB(),
            "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
            "Stacking Meta-Model": stacking
        }

    # ---------------------------
    # Preprocessing + feature selection
    # ---------------------------
    def preprocess(self, df, target_hint):
        drop_like = [c for c in df.columns if c.lower() in ["id","name","subjectid","patientid","caseid"]]
        df = df.drop(columns=drop_like, errors="ignore")

        target = guess_target(df, target_hint)
        if df[target].dtype == 'object':
            le = LabelEncoder()
            df[target] = le.fit_transform(df[target].astype(str))

        X = df.drop(columns=[target])
        y = df[target]

        num_cols = X.select_dtypes(include=[np.number]).columns
        cat_cols = [c for c in X.columns if c not in num_cols]
        X[num_cols] = X[num_cols].fillna(X[num_cols].median())
        for c in cat_cols:
            if X[c].isna().any():
                X[c] = X[c].fillna(X[c].mode()[0])
        X = pd.get_dummies(X, drop_first=True)

        # Feature selection
        if self.n_features < X.shape[1]:
            selector = SelectKBest(score_func=f_classif, k=self.n_features)
            X = selector.fit_transform(X, y)

        return X, y

    # ---------------------------
    # Train/Test split + scaling + SMOTE
    # ---------------------------
    def prepare_train(self, X, y):
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, stratify=y if y.nunique()>1 else None, random_state=42
        )
        if self.balance:
            sm = SMOTE(random_state=42)
            X_train, y_train = sm.fit_resample(X_train, y_train)

        scaler = StandardScaler(with_mean=False)
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        return X_train, X_val, y_train, y_val, scaler

    # ---------------------------
    # Train all models + hyperparameter tuning
    # ---------------------------
    def fit(self):
        for ds_name, (path, target_hint) in self.datasets.items():
            print(f"\n====================\n📌 Training {ds_name}\n====================")
            df = pd.read_csv(path)
            X, y = self.preprocess(df, target_hint)
            X_train, X_val, y_train, y_val, scaler = self.prepare_train(X, y)
            self.scalers[ds_name] = scaler

            models = self.get_models()
            results = []

            for name, model in models.items():
                try:
                    start = time.time()
                    if self.hyperparam_tuning and name in ["Random Forest", "XGBoost"]:
                        if name=="Random Forest":
                            params = {'n_estimators':[50,100], 'max_depth':[None,5,10]}
                        else:
                            params = {'max_depth':[3,5,7],'learning_rate':[0.01,0.1]}
                        grid = GridSearchCV(model, param_grid=params, cv=3, scoring='balanced_accuracy')
                        grid.fit(X_train, y_train)
                        model = grid.best_estimator_
                    else:
                        model.fit(X_train, y_train)
                    elapsed = round(time.time() - start, 3)

                    y_pred = model.predict(X_val)
                    roc = safe_roc_auc(y_val, model, X_val)
                    acc = accuracy_score(y_val, y_pred)
                    bal_acc = balanced_accuracy_score(y_val, y_pred)
                    f1 = f1_score(y_val, y_pred, average='weighted')
                    prec = precision_score(y_val, y_pred, average='weighted', zero_division=0)
                    rec = recall_score(y_val, y_pred, average='weighted', zero_division=0)

                    results.append([name, acc, bal_acc, roc, f1, prec, rec, elapsed])
                except Exception as e:
                    results.append([name, "ERR"]*7 + [str(e)])

            df_results = pd.DataFrame(results, columns=["Model","Accuracy","Balanced Accuracy","ROC-AUC","F1","Precision","Recall","Time"])
            display(df_results)

            # Select best model
            best_row = df_results[df_results["Balanced Accuracy"]!="ERR"].sort_values("Balanced Accuracy", ascending=False).iloc[0]
            self.best_models[ds_name] = models[best_row["Model"]]
            self.best_models[ds_name].fit(X_train, y_train)
            print(f"✅ Best model for {ds_name}: {best_row['Model']} with Balanced Accuracy={best_row['Balanced Accuracy']}")

            self.results[ds_name] = df_results

    # ---------------------------
    # Predict test data
    # ---------------------------
    def predict_test(self, ds_name, test_path, save_csv=True):
        if ds_name not in self.best_models:
            print(f"No trained model for {ds_name}")
            return None
        df_test = pd.read_csv(test_path)
        drop_like = [c for c in df_test.columns if c.lower() in ["id","name","subjectid","patientid","caseid"]]
        df_test = df_test.drop(columns=drop_like, errors="ignore")
        X_test = pd.get_dummies(df_test, drop_first=True)

        # Align columns
        train_cols = self.scalers[ds_name].mean_.shape[0] if hasattr(self.scalers[ds_name], 'mean_') else X_test.shape[1]
        X_test = X_test.reindex(columns=range(train_cols), fill_value=0)

        X_test_scaled = self.scalers[ds_name].transform(X_test)
        y_pred = self.best_models[ds_name].predict(X_test_scaled)
        if save_csv:
            pd.DataFrame({"ID": range(1,len(y_pred)+1), "Prediction": y_pred}).to_csv(f"{ds_name}_test_predictions.csv", index=False)
            print(f"Test predictions saved as '{ds_name}_test_predictions.csv'")
        return y_pred

    # ---------------------------
    # Save models
    # ---------------------------
    def save_models(self, folder_path="./"):
        for ds_name, model in self.best_models.items():
            path = f"{folder_path}{ds_name}_best_model.pkl"
            joblib.dump(model, path)
            print(f"{ds_name} model saved as {path}")

# ============================================
# Example usage for Neurodivergent Datasets
# ============================================

datasets = {
    "Autism": ("/content/drive/MyDrive/Neurodivergent/autism_screening.csv", "Class/ASD"),
    "Alzheimer": ("/content/drive/MyDrive/Neurodivergent/alzheimers_disease_data.csv", "Class"),
    "ALS": ("/content/drive/MyDrive/Neurodivergent/Minsk2020_ALS_dataset.csv", "diagnosis"),
    "Parkinson": ("/content/drive/MyDrive/Neurodivergent/Parkinsson disease.csv", "status")
}

automl = AutoML(datasets=datasets, n_features=20, balance=True, hyperparam_tuning=False)
automl.fit()

# Example: Predict test data for Autism
# automl.predict_test("Autism", "/content/drive/MyDrive/Neurodivergent/autism_test.csv")
# Save all models
# automl.save_models()



📌 Training Autism


,Model,Accuracy,Balanced Accuracy,ROC-AUC,F1,Precision,Recall,Time
0,Logistic Regression,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.016
1,Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.264
2,SVM,0.978723,0.968830,0.997956,0.978633,0.978668,0.978723,0.070
3,KNN,0.921986,0.913388,0.977133,0.922891,0.924767,0.921986,0.001
4,Naive Bayes,0.914894,0.875319,0.925268,0.913340,0.913677,0.914894,0.002
5,XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.051
6,Stacking Meta-Model,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.314


✅ Best model for Autism: Logistic Regression with Balanced Accuracy=1.0

📌 Training Alzheimer


,Model,Accuracy,Balanced Accuracy,ROC-AUC,F1,Precision,Recall,Time
0,Logistic Regression,0.576744,0.491190,0.492549,0.638451,0.749364,0.576744,0.023
1,Random Forest,0.855814,0.500000,0.461299,0.789322,0.732418,0.855814,0.939
2,SVM,0.732558,0.461518,0.477187,0.733446,0.734338,0.732558,1.892
3,KNN,0.562791,0.529979,0.522550,0.627801,0.766832,0.562791,0.001
4,Naive Bayes,0.576744,0.477779,0.470678,0.637946,0.743578,0.576744,0.002
5,XGBoost,0.837209,0.489130,0.477603,0.779982,0.730078,0.837209,0.397
6,Stacking Meta-Model,0.832558,0.486413,0.503419,0.777618,0.729480,0.832558,15.454


✅ Best model for Alzheimer: KNN with Balanced Accuracy=0.5299789621318374

📌 Training ALS


,Model,Accuracy,Balanced Accuracy,ROC-AUC,F1,Precision,Recall,Time
0,Logistic Regression,0.692308,0.690476,0.857143,0.692308,0.692308,0.692308,0.013
1,Random Forest,0.846154,0.845238,0.904762,0.846154,0.846154,0.846154,0.214
2,SVM,0.846154,0.833333,0.904762,0.840385,0.880342,0.846154,0.002
3,KNN,0.692308,0.702381,0.797619,0.688645,0.719231,0.692308,0.001
4,Naive Bayes,0.846154,0.845238,0.952381,0.846154,0.846154,0.846154,0.001
5,XGBoost,0.846154,0.845238,0.904762,0.846154,0.846154,0.846154,0.020
6,Stacking Meta-Model,0.846154,0.845238,0.928571,0.846154,0.846154,0.846154,1.267


✅ Best model for ALS: Random Forest with Balanced Accuracy=0.8452380952380952

📌 Training Parkinson


,Model,Accuracy,Balanced Accuracy,ROC-AUC,F1,Precision,Recall,Time
0,Logistic Regression,0.769231,0.779310,0.920690,0.781422,0.818376,0.769231,0.028
1,Random Forest,0.923077,0.915517,0.977586,0.924234,0.926823,0.923077,0.234
2,SVM,0.692308,0.793103,0.858621,0.709866,0.860140,0.692308,0.014
3,KNN,0.897436,0.931034,0.994828,0.902184,0.926740,0.897436,0.001
4,Naive Bayes,0.666667,0.743103,0.868966,0.686799,0.812169,0.666667,0.001
5,XGBoost,0.948718,0.932759,0.986207,0.948718,0.948718,0.948718,0.115
6,Stacking Meta-Model,0.948718,0.932759,0.996552,0.948718,0.948718,0.948718,2.445


✅ Best model for Parkinson: Stacking Meta-Model with Balanced Accuracy=0.9327586206896552


In [ ]:
# =============================================
# Federated Learning for Neurodivergent Datasets
# =============================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy
import re, difflib, warnings, time

warnings.filterwarnings("ignore")

# ===========================
# Helper Functions
# ===========================
def guess_target(df, preferred=None):
    cols = list(df.columns)
    norm = lambda s: re.sub(r'[^a-z0-9]+', '', str(s).lower())
    if preferred and preferred in cols:
        return preferred
    if preferred:
        norm_pref = norm(preferred)
        for c in cols:
            if norm(c) == norm_pref:
                return c
        close = difflib.get_close_matches(preferred, cols, n=1, cutoff=0.6)
        if close:
            return close[0]
    kw = ['class', 'asd', 'diagnosis', 'label', 'target', 'status', 'group', 'outcome', 'response', 'disease', 'result']
    for c in cols:
        if any(k in norm(c) for k in kw):
            return c
    return cols[-1]

def preprocess_data(path, target_hint):
    df = pd.read_csv(path)

    # Drop ID-like columns
    drop_like = [c for c in df.columns if c.lower() in ["id","name","subjectid","patientid","caseid"]]
    df = df.drop(columns=drop_like, errors="ignore")

    # Auto-detect target
    target = guess_target(df, target_hint)
    if df[target].dtype == 'object':
        le = LabelEncoder()
        df[target] = le.fit_transform(df[target].astype(str))

    X = df.drop(columns=[target])
    y = df[target]

    # Handle missing values
    num_cols = X.select_dtypes(include=[np.number]).columns
    cat_cols = [c for c in X.columns if c not in num_cols]
    X[num_cols] = X[num_cols].fillna(X[num_cols].median())
    for c in cat_cols:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].mode()[0])

    X = pd.get_dummies(X, drop_first=True)

    # Scale
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y.values

# ===========================
# PyTorch Model
# ===========================
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=2):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# ===========================
# Federated Averaging Function
# ===========================
def federated_avg(global_model, client_models, client_data_sizes):
    global_dict = global_model.state_dict()
    for key in global_dict.keys():
        global_dict[key] = torch.stack([client_models[i].state_dict()[key].float() * client_data_sizes[i]
                                        for i in range(len(client_models))], 0).sum(0) / sum(client_data_sizes)
    global_model.load_state_dict(global_dict)
    return global_model

# ===========================
# Training Function
# ===========================
def train_local(model, X, y, epochs=5, lr=0.01):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()
    return model

# ===========================
# Evaluation Function
# ===========================
def evaluate(model, X, y):
    model.eval()
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    with torch.no_grad():
        outputs = model(X_tensor)
        preds = torch.argmax(outputs, dim=1)
        acc = (preds == y_tensor).float().mean().item()
    return acc

# ===========================
# Neurodivergent Datasets Paths
# ===========================
datasets = {
    "Autism": ("/content/drive/MyDrive/Neurodivergent/autism_screening.csv", "Class/ASD"),
    "Alzheimer": ("/content/drive/MyDrive/Neurodivergent/alzheimers_disease_data.csv", "Class"),
    "ALS": ("/content/drive/MyDrive/Neurodivergent/Minsk2020_ALS_dataset.csv", "diagnosis"),
    "Parkinson": ("/content/drive/MyDrive/Neurodivergent/Parkinsson disease.csv", "status")
}

# ===========================
# Federated Learning Simulation
# ===========================
clients_data = {}
clients_sizes = {}
max_input_dim = 0

# Preprocess datasets and determine max features
for name, (path, target_hint) in datasets.items():
    X, y = preprocess_data(path, target_hint)
    clients_data[name] = (X, y)
    clients_sizes[name] = X.shape[0]
    if X.shape[1] > max_input_dim:
        max_input_dim = X.shape[1]

# Pad smaller datasets to match max_input_dim
for name in clients_data:
    X, y = clients_data[name]
    if X.shape[1] < max_input_dim:
        padding_needed = max_input_dim - X.shape[1]
        X_padded = np.pad(X, ((0, 0), (0, padding_needed)), 'constant')
        clients_data[name] = (X_padded, y)


# Initialize global model
output_dim = max([len(np.unique(clients_data[name][1])) for name in datasets])  # max num of classes
global_model = SimpleNN(input_dim=max_input_dim, output_dim=output_dim)

rounds = 5
local_epochs = 5

for r in range(rounds):
    print(f"\n--- Federated Round {r+1} ---")
    client_models = []
    for name, (X, y) in clients_data.items():
        local_model = deepcopy(global_model)
        local_model = train_local(local_model, X, y, epochs=local_epochs)
        acc = evaluate(local_model, X, y)
        print(f"{name} local accuracy: {acc:.4f}")
        client_models.append(local_model)

    # Federated averaging
    global_model = federated_avg(global_model, client_models, list(clients_sizes.values()))

# ===========================
# Evaluate Global Model
# ===========================
print("\n--- Global Model Evaluation ---")
for name, (X, y) in clients_data.items():
    acc = evaluate(global_model, X, y)
    print(f"{name} accuracy: {acc:.4f}")


--- Federated Round 1 ---
Autism local accuracy: 0.9418
Alzheimer local accuracy: 0.8557
ALS local accuracy: 0.9688
Parkinson local accuracy: 0.8821

--- Federated Round 2 ---
Autism local accuracy: 0.9432
Alzheimer local accuracy: 0.8557
ALS local accuracy: 0.9844
Parkinson local accuracy: 0.7282

--- Federated Round 3 ---
Autism local accuracy: 0.9347
Alzheimer local accuracy: 0.8562
ALS local accuracy: 0.9844
Parkinson local accuracy: 0.6308

--- Federated Round 4 ---
Autism local accuracy: 0.9659
Alzheimer local accuracy: 0.8567
ALS local accuracy: 1.0000
Parkinson local accuracy: 0.7487

--- Federated Round 5 ---
Autism local accuracy: 0.9773
Alzheimer local accuracy: 0.8604
ALS local accuracy: 1.0000
Parkinson local accuracy: 0.8308

--- Global Model Evaluation ---
Autism accuracy: 0.8679
Alzheimer accuracy: 0.8646
ALS accuracy: 0.5938
Parkinson accuracy: 0.3231


In [ ]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
import joblib

# Load
saved_obj = joblib.load("/content/als_results/als_best_pipeline_lr.joblib")
best_pipeline = saved_obj["pipeline"]
meta_learner = saved_obj["meta_learner"]

# View pipeline steps
print("Best pipeline steps:")
for name, step in best_pipeline.steps:
    print(f" - {name}: {step}")

# View meta-learner
print("\nMeta-learner:")
print(meta_learner)


Best pipeline steps:
 - pre: ColumnTransformer(sparse_threshold=0,
                  transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  IterativeImputer(random_state=42)),
                                                 ('scaler', RobustScaler())]),
                                 ['Age', 'J1_a', 'J3_a', 'J5_a', 'J55_a',
                                  'S1_a', 'S3_a', 'S5_a', 'S11_a', 'S55_a',
                                  'DPF_a', 'PFR_a', 'PPE_a', 'PVI_a', 'HNR_a',
                                  'GNEa_{\\mu}', 'GNEa_{\\sigma}', 'Ha(1)_{mu}',
                                  'Ha(2)_{mu}', 'Ha(3)_{mu}', 'Ha(4)_{mu}',
                                  'Ha(5)_{mu}', 'Ha(6)_{mu}', 'Ha(7)_{mu}',
                                  'Ha(8)_{mu}', 'Ha(1)_{sd}', 'Ha(2)_{sd}',
                                  'Ha(3)_{sd}', 'Ha(4)_{sd}', 'Ha(5)_{sd}', ...]),
                                ('cat',
      

In [ ]:
# If you still have nas_model in memory
nas_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 224)            │        30,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224)            │           896 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 192)            │        43,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 192)            │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 224,229 (875.90 KB)

 Trainable params: 74,465 (290.88 KB)

 Non-trainable params: 832 (3.25 KB)

 Optimizer params: 148,932 (581.77 KB)

In [ ]:
pip install tensorflow

In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Multi-Dataset ALS pipeline: tuning, NAS, evaluation, meta-learner, save artifacts
import os, time
import numpy as np
import pandas as pd

from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, roc_auc_score,
    f1_score, precision_score, recall_score, confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# Optional: imblearn for SMOTE inside pipelines
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
    HAS_IMBLEARN = True
except Exception:
    HAS_IMBLEARN = False

# ---------------------------
# Configuration
# ---------------------------
DATASET_CONFIGS = {
    "dataset_1": {
        "path": "/content/drive/MyDrive/Neurodivergent/Minsk2020_ALS_dataset.csv",
        "name": "Dataset_1"
    },
    "dataset_2": {
        "path": "/content/drive/MyDrive/Neurodivergent/Parkinsson disease.csv",
        "name": "Dataset_2"
    },
    "dataset_3": {
        "path": "/content/drive/MyDrive/Neurodivergent/alzheimers_disease_data.csv",
        "name": "Dataset_3"
    }
}

RANDOM_STATE = 42
BASE_RESULTS_DIR = "/content/multi_dataset_results"
os.makedirs(BASE_RESULTS_DIR, exist_ok=True)

# ---------------------------
# Helper: OneHotEncoder version-safe
# ---------------------------
def get_onehot_encoder(**kwargs):
    try:
        # sklearn >= 1.2
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False, **kwargs)
    except TypeError:
        # older sklearn
        return OneHotEncoder(handle_unknown="ignore", sparse=False, **kwargs)

# ---------------------------
# Global NAS Architecture Storage
# ---------------------------
SHARED_NAS_ARCHITECTURE = None

# ---------------------------
# Expanded NAS using keras-tuner (with architecture sharing)
# ---------------------------
def run_keras_nas_search(X_train, y_train, max_trials=6, epochs=8, batch_size=32, verbose=0):
    """
    Use keras-tuner to search MLP architectures on the first dataset.
    Returns the best keras.Model instance and architecture config.
    """
    try:
        import tensorflow as tf
        from tensorflow import keras
    except Exception:
        print("TensorFlow not available — skipping NAS.")
        return None, None

    # try import keras_tuner
    kt = None
    try:
        import keras_tuner as kt
    except Exception:
        try:
            import kerastuner as kt
        except Exception:
            kt = None

    if kt is None:
        print("Keras Tuner not available — skipping NAS.")
        return None, None

    input_shape = int(X_train.shape[1])
    n_classes = len(np.unique(y_train))

    def build_model(hp):
        model = keras.Sequential()
        num_layers = hp.Int("num_layers", 1, 4)
        activation = hp.Choice("activation", ["relu", "tanh"])
        learning_rate = hp.Choice("lr", [1e-2, 1e-3, 1e-4])

        # Store architecture config for reuse
        arch_config = {
            "num_layers": num_layers,
            "activation": activation,
            "learning_rate": learning_rate,
            "layers": []
        }

        for i in range(num_layers):
            units = hp.Int(f"units_{i}", 32, 256, step=32)
            use_batchnorm = hp.Boolean(f"batchnorm_{i}")
            use_dropout = hp.Boolean(f"dropout_{i}")
            dropout_rate = hp.Float(f"dropout_rate_{i}", 0.1, 0.5, step=0.1) if use_dropout else 0.0

            arch_config["layers"].append({
                "units": units,
                "use_batchnorm": use_batchnorm,
                "use_dropout": use_dropout,
                "dropout_rate": dropout_rate
            })

            if i == 0:
                model.add(keras.layers.Dense(units, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(keras.layers.Dense(units, activation=activation))

            if use_batchnorm:
                model.add(keras.layers.BatchNormalization())
            if use_dropout:
                model.add(keras.layers.Dropout(rate=dropout_rate))

        # output layer
        if n_classes > 2:
            model.add(keras.layers.Dense(n_classes, activation="softmax"))
            loss = "sparse_categorical_crossentropy"
        else:
            model.add(keras.layers.Dense(1, activation="sigmoid"))
            loss = "binary_crossentropy"

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss=loss,
            metrics=["accuracy"]
        )

        # Store config in model for later extraction
        model._arch_config = arch_config
        return model

    tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=max_trials, overwrite=True,
                            directory="nas_search", project_name="multi_dataset_nas")
    tuner.search(X_train, y_train, validation_split=0.15, epochs=epochs, batch_size=batch_size, verbose=verbose)
    best_models = tuner.get_best_models(num_models=1)

    if best_models:
        best_model = best_models[0]
        # Extract architecture config
        arch_config = getattr(best_model, '_arch_config', None)
        return best_model, arch_config
    return None, None

def build_model_from_architecture(arch_config, input_shape, n_classes):
    """
    Build a Keras model using the shared architecture configuration.
    """
    try:
        import tensorflow as tf
        from tensorflow import keras
    except Exception:
        return None

    if arch_config is None:
        return None

    model = keras.Sequential()

    for i, layer_config in enumerate(arch_config["layers"]):
        if i == 0:
            model.add(keras.layers.Dense(
                layer_config["units"],
                activation=arch_config["activation"],
                input_shape=(input_shape,)
            ))
        else:
            model.add(keras.layers.Dense(
                layer_config["units"],
                activation=arch_config["activation"]
            ))

        if layer_config["use_batchnorm"]:
            model.add(keras.layers.BatchNormalization())
        if layer_config["use_dropout"]:
            model.add(keras.layers.Dropout(rate=layer_config["dropout_rate"]))

    # output layer
    if n_classes > 2:
        model.add(keras.layers.Dense(n_classes, activation="softmax"))
        loss = "sparse_categorical_crossentropy"
    else:
        model.add(keras.layers.Dense(1, activation="sigmoid"))
        loss = "binary_crossentropy"

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=arch_config["learning_rate"]),
        loss=loss,
        metrics=["accuracy"]
    )

    return model

# ---------------------------
# Keras wrapper
# ---------------------------
class KerasWrapper:
    def __init__(self, model, epochs=10, batch_size=32, verbose=0):
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self._is_fitted = False

    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        self._is_fitted = True
        return self

    def predict(self, X):
        proba = self.model.predict(X, batch_size=self.batch_size, verbose=0)
        proba = np.asarray(proba)
        if proba.ndim == 1 or proba.shape[1] == 1:
            # binary
            p = proba.ravel()
            return (p > 0.5).astype(int)
        else:
            return np.argmax(proba, axis=1)

    def predict_proba(self, X):
        proba = self.model.predict(X, batch_size=self.batch_size, verbose=0)
        proba = np.asarray(proba)
        if proba.ndim == 1 or proba.shape[1] == 1:
            p = proba.ravel()
            return np.vstack([1 - p, p]).T
        else:
            return proba

# ---------------------------
# Dataset processing functions
# ---------------------------
def detect_target(df):
    kws = ["diagnosis", "label", "class", "target", "outcome", "result", "status"]
    for c in df.columns:
        if any(k in c.lower() for k in kws):
            return c
    return df.columns[-1]

def load_and_preprocess_dataset(dataset_path, dataset_name):
    """Load and preprocess a single dataset."""
    print(f"\n{'='*50}")
    print(f"Processing {dataset_name}")
    print(f"{'='*50}")

    # Load dataset
    df = pd.read_csv(dataset_path)
    print(f"Loaded {dataset_name}:", df.shape)

    # Drop id-like columns
    drop_like = [c for c in df.columns if c.lower() in ["id", "name", "subjectid", "patientid", "caseid"]]
    df = df.drop(columns=drop_like, errors="ignore")

    # Detect target column
    target_col = detect_target(df)
    print(f"Target detected for {dataset_name}:", target_col)

    # Ensure numeric labels
    if df[target_col].dtype == "object":
        le = LabelEncoder()
        df[target_col] = le.fit_transform(df[target_col].astype(str))
    elif not np.issubdtype(df[target_col].dtype, np.number):
        df[target_col] = pd.factorize(df[target_col])[0]

    X = df.drop(columns=[target_col])
    y = df[target_col].values

    return X, y, target_col

# ---------------------------
# Model definitions
# ---------------------------
def get_models_and_spaces():
    """Get candidate models and their parameter spaces."""
    return {
        "rf": (
            RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
            {"clf__n_estimators": [100, 200, 400], "clf__max_depth": [None, 6, 12, 20], "clf__min_samples_split": [2, 5, 10]}
        ),
        "xgb": (
            XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=RANDOM_STATE, n_jobs=1),
            {"clf__n_estimators": [100, 200, 400], "clf__max_depth": [3, 6, 10], "clf__learning_rate": [0.01, 0.05, 0.1]}
        ),
        "svc": (
            SVC(probability=True, random_state=RANDOM_STATE),
            {"clf__C": [0.1, 1, 10], "clf__gamma": ["scale", "auto"]}
        ),
        "knn": (
            KNeighborsClassifier(),
            {"clf__n_neighbors": [3, 5, 7, 9], "clf__weights": ["uniform", "distance"]}
        ),
        "lr": (
            LogisticRegression(max_iter=2000),
            {"clf__C": [0.01, 0.1, 1, 10]}
        ),
        "nb": (
            GaussianNB(),
            {"clf__var_smoothing": [1e-9, 1e-8, 1e-7]}
        )
    }

# ---------------------------
# Training and evaluation functions
# ---------------------------
def create_preprocessing_pipeline(X_df):
    """Create preprocessing pipeline for a dataset."""
    num_cols = X_df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_df.columns if c not in num_cols]

    print(f"Numeric cols: {len(num_cols)}, Categorical cols: {len(cat_cols)}")

    numeric_transformer = Pipeline([
        ("imputer", IterativeImputer(random_state=RANDOM_STATE, max_iter=10)),
        ("scaler", RobustScaler())
    ])
    categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", get_onehot_encoder())
    ])

    preprocessor = ColumnTransformer([
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ], remainder="drop", sparse_threshold=0)

    return preprocessor

def tune_and_evaluate(model_name, estimator, param_dist, X_df, y_arr, preprocessor, n_iter=12, cv_splits=3):
    """Tune and evaluate a single model."""
    use_pipeline_cls = ImbPipeline if HAS_IMBLEARN else Pipeline

    # Build pipeline
    pipe_steps = [("pre", preprocessor)]
    if HAS_IMBLEARN:
        from collections import Counter
        counts = Counter(y_arr)
        if len(counts) > 1:
            imbalance = max(counts.values()) / min(counts.values())
        else:
            imbalance = 1.0
        if imbalance > 1.5:
            pipe_steps.append(("smote", SMOTE(random_state=RANDOM_STATE)))

    k_default = min(50, max(5, int(max(5, X_df.shape[1] * 0.5))))
    pipe_steps.append(("select", SelectKBest(score_func=f_classif, k=k_default)))
    pipe_steps.append(("clf", estimator))
    pipeline = use_pipeline_cls(steps=pipe_steps)

    rskf = RepeatedStratifiedKFold(n_splits=cv_splits, n_repeats=2, random_state=RANDOM_STATE)
    search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=n_iter,
                                cv=rskf, scoring="balanced_accuracy", n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
    start = time.time()
    search.fit(X_df, y_arr)
    elapsed = time.time() - start
    best = search.best_estimator_
    print(f"[{model_name}] best score: {search.best_score_:.4f} time: {elapsed:.1f}s")
    return best, search

def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    """Evaluate a single model."""
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = None
    try:
        y_proba = model.predict_proba(X_test)
    except Exception:
        y_proba = None

    metrics = {
        "model": name,
        "accuracy": accuracy_score(y_test, y_pred),
        "balanced_accuracy": balanced_accuracy_score(y_test, y_pred),
        "f1_weighted": f1_score(y_test, y_pred, average="weighted"),
        "precision_weighted": precision_score(y_test, y_pred, average="weighted", zero_division=0),
        "recall_weighted": recall_score(y_test, y_pred, average="weighted", zero_division=0),
    }

    if y_proba is not None:
        try:
            if len(np.unique(y_test)) == 2:
                metrics["roc_auc"] = roc_auc_score(y_test, y_proba[:, 1])
            else:
                metrics["roc_auc"] = roc_auc_score(y_test, y_proba, multi_class="ovr")
        except Exception:
            metrics["roc_auc"] = None
    else:
        metrics["roc_auc"] = None

    print(f"\n=== {name.upper()} ===")
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification report:\n", classification_report(y_test, y_pred))
    return metrics

def process_single_dataset(dataset_key, dataset_config, is_first_dataset=False):
    """Process a single dataset with the NAS architecture."""
    global SHARED_NAS_ARCHITECTURE

    dataset_name = dataset_config["name"]
    dataset_path = dataset_config["path"]

    # Create results directory for this dataset
    results_dir = os.path.join(BASE_RESULTS_DIR, dataset_key)
    os.makedirs(results_dir, exist_ok=True)

    # Load and preprocess dataset
    X, y, target_col = load_and_preprocess_dataset(dataset_path, dataset_name)

    # Train/holdout split
    X_train_df, X_hold_df, y_train, y_hold = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE,
        stratify=y if len(np.unique(y)) > 1 else None
    )

    # Create preprocessing pipeline
    preprocessor = create_preprocessing_pipeline(X_train_df)

    # Get models and parameter spaces
    models_and_spaces = get_models_and_spaces()

    # Tune traditional ML models
    print(f"\nTuning traditional ML models for {dataset_name}...")
    tuned_models = {}
    for name, (est, space) in models_and_spaces.items():
        try:
            best_est, search_obj = tune_and_evaluate(name, est, space, X_train_df, y_train, preprocessor)
            tuned_models[name] = best_est
        except Exception as e:
            print(f"Failed tuning {name}: {e}")

    # Handle NAS model
    nas_model = None
    if is_first_dataset:
        # First dataset: perform NAS search
        print(f"\nPerforming NAS search on {dataset_name}...")
        try:
            X_train_pre = preprocessor.fit_transform(X_train_df)
            X_train_pre = np.asarray(X_train_pre, dtype=np.float32)
            print(f"Preprocessed training data shape: {X_train_pre.shape}")
            print(f"Number of classes: {len(np.unique(y_train))}")

            nas_model, SHARED_NAS_ARCHITECTURE = run_keras_nas_search(
                X_train_pre, y_train, max_trials=6, epochs=8, batch_size=32, verbose=0
            )
            if nas_model is not None and SHARED_NAS_ARCHITECTURE is not None:
                print(f"NAS discovered architecture for {dataset_name}")
                print("Architecture details:", SHARED_NAS_ARCHITECTURE)
                nas_model.summary()
            else:
                print("NAS failed or not available - checking individual components:")
                print(f"  nas_model is None: {nas_model is None}")
                print(f"  SHARED_NAS_ARCHITECTURE is None: {SHARED_NAS_ARCHITECTURE is None}")
        except Exception as e:
            print(f"NAS attempt failed: {e}")
            import traceback
            traceback.print_exc()
    else:
        # Subsequent datasets: use shared architecture
        if SHARED_NAS_ARCHITECTURE is not None:
            print(f"\nApplying shared NAS architecture to {dataset_name}...")
            print("Using architecture:", SHARED_NAS_ARCHITECTURE)
            try:
                X_train_pre = preprocessor.fit_transform(X_train_df)
                X_train_pre = np.asarray(X_train_pre, dtype=np.float32)
                input_shape = X_train_pre.shape[1]
                n_classes = len(np.unique(y_train))

                print(f"Building NAS model with input_shape={input_shape}, n_classes={n_classes}")
                nas_model = build_model_from_architecture(SHARED_NAS_ARCHITECTURE, input_shape, n_classes)
                if nas_model is not None:
                    print(f"Successfully built NAS model for {dataset_name}")
                    nas_model.summary()
                else:
                    print("Failed to build NAS model from shared architecture")
            except Exception as e:
                print(f"Failed to apply shared NAS architecture: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("No shared NAS architecture available")

    # Add NAS model to tuned_models if available
    if nas_model is not None:
        keras_wrapper = KerasWrapper(nas_model, epochs=8, batch_size=32, verbose=0)
        nas_pipeline = Pipeline([("pre", preprocessor), ("keras", keras_wrapper)])
        tuned_models["nas"] = nas_pipeline

    # Evaluate all models
    print(f"\nEvaluating all models for {dataset_name}...")
    all_results = []
    for name, model in tuned_models.items():
        try:
            metrics = evaluate_model(name, model, X_train_df, y_train, X_hold_df, y_hold)
            all_results.append(metrics)
        except Exception as e:
            print(f"Evaluation failed for {name}: {e}")

    # Create results summary
    if all_results:
        results_df = pd.DataFrame(all_results).sort_values(by="balanced_accuracy", ascending=False).reset_index(drop=True)
        print(f"\nSummary of all models for {dataset_name}:")
        print(results_df)

        # Save results
        results_df.to_csv(os.path.join(results_dir, f"{dataset_key}_results.csv"), index=False)

        # Get best model
        best_row = results_df.iloc[0]
        best_model_name = best_row["model"]
        print(f"\n🔥 Best model for {dataset_name} is: {best_model_name}")
        print(f"Best balanced accuracy: {best_row['balanced_accuracy']:.4f}")

        # Save best model
        import joblib
        try:
            best_pipeline = tuned_models[best_model_name]
            if best_model_name == "nas" and nas_model is not None:
                # Save Keras model separately
                keras_save_path = os.path.join(results_dir, f"{dataset_key}_nas_best_model.keras")
                nas_model.save(keras_save_path)
                joblib.dump({"preprocessor": preprocessor, "note": "NAS Keras model saved separately."},
                           os.path.join(results_dir, f"{dataset_key}_preprocessor_for_nas.joblib"))
                print(f"Saved NAS Keras model to: {keras_save_path}")
            else:
                save_path = os.path.join(results_dir, f"{dataset_key}_best_{best_model_name}.joblib")
                joblib.dump(best_pipeline, save_path)
                print(f"Saved best sklearn pipeline to: {save_path}")
        except Exception as e:
            print(f"Saving best model failed: {e}")

        return {
            "dataset_name": dataset_name,
            "results_df": results_df,
            "best_model_name": best_model_name,
            "best_metrics": best_row,
            "tuned_models": tuned_models,
            "preprocessor": preprocessor
        }
    else:
        print(f"No models evaluated successfully for {dataset_name}")
        return None

# ---------------------------
# Main execution
# ---------------------------
def main():
    """Main function to process all datasets."""
    print("Starting Multi-Dataset ALS Pipeline with NAS")
    print(f"Processing {len(DATASET_CONFIGS)} datasets")

    all_dataset_results = {}

    # Process each dataset
    for i, (dataset_key, dataset_config) in enumerate(DATASET_CONFIGS.items()):
        is_first = (i == 0)  # First dataset performs NAS search

        try:
            result = process_single_dataset(dataset_key, dataset_config, is_first_dataset=is_first)
            if result:
                all_dataset_results[dataset_key] = result
        except Exception as e:
            print(f"Failed to process {dataset_key}: {e}")
            continue

    # Cross-dataset analysis
    if all_dataset_results:
        print(f"\n{'='*60}")
        print("CROSS-DATASET ANALYSIS")
        print(f"{'='*60}")

        # Compare best models across datasets
        comparison_data = []
        for dataset_key, result in all_dataset_results.items():
            comparison_data.append({
                "Dataset": result["dataset_name"],
                "Best_Model": result["best_model_name"],
                "Balanced_Accuracy": result["best_metrics"]["balanced_accuracy"],
                "F1_Weighted": result["best_metrics"]["f1_weighted"],
                "ROC_AUC": result["best_metrics"].get("roc_auc", "N/A")
            })

        comparison_df = pd.DataFrame(comparison_data)
        print("\nCross-dataset comparison:")
        print(comparison_df)

        # Save cross-dataset comparison
        comparison_df.to_csv(os.path.join(BASE_RESULTS_DIR, "cross_dataset_comparison.csv"), index=False)

        # Save shared NAS architecture if available
        if SHARED_NAS_ARCHITECTURE is not None:
            import joblib
            try:
                joblib.dump(SHARED_NAS_ARCHITECTURE, os.path.join(BASE_RESULTS_DIR, "shared_nas_architecture.joblib"))
                print(f"\nSaved shared NAS architecture to: {BASE_RESULTS_DIR}/shared_nas_architecture.joblib")
            except Exception as e:
                print(f"Failed to save shared NAS architecture: {e}")

        # Model performance analysis
        model_performance = {}
        for dataset_key, result in all_dataset_results.items():
            for _, row in result["results_df"].iterrows():
                model_name = row["model"]
                if model_name not in model_performance:
                    model_performance[model_name] = []
                model_performance[model_name].append(row["balanced_accuracy"])

        print("\nAverage model performance across datasets:")
        for model, scores in model_performance.items():
            avg_score = np.mean(scores)
            std_score = np.std(scores)
            print(f"{model}: {avg_score:.4f} ± {std_score:.4f}")

    print(f"\nPipeline completed. Results saved in: {BASE_RESULTS_DIR}")

if __name__ == "__main__":
    main()

Starting Multi-Dataset ALS Pipeline with NAS
Processing 3 datasets

Processing Dataset_1
Loaded Dataset_1: (64, 135)
Target detected for Dataset_1: Diagnosis (ALS)
Numeric cols: 132, Categorical cols: 1

Tuning traditional ML models for Dataset_1...
[rf] best score: 0.7569 time: 83.9s


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [17:20:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=12. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[xgb] best score: 0.7095 time: 36.7s
[svc] best score: 0.7118 time: 17.4s


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 8 is smaller than n_iter=12. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[knn] best score: 0.6956 time: 22.8s


...


=== LR ===
Confusion matrix:
 [[220  58]
 [ 23 129]]
Classification report:
               precision    recall  f1-score   support

           0       0.91      0.79      0.84       278
           1       0.69      0.85      0.76       152

    accuracy                           0.81       430
   macro avg       0.80      0.82      0.80       430
weighted avg       0.83      0.81      0.82       430



/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [32] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



=== NB ===
Confusion matrix:
 [[215  63]
 [ 30 122]]
Classification report:
               precision    recall  f1-score   support

           0       0.88      0.77      0.82       278
           1       0.66      0.80      0.72       152

    accuracy                           0.78       430
   macro avg       0.77      0.79      0.77       430
weighted avg       0.80      0.78      0.79       430



/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(



=== NAS ===
Confusion matrix:
 [[242  36]
 [ 37 115]]
Classification report:
               precision    recall  f1-score   support

           0       0.87      0.87      0.87       278
           1       0.76      0.76      0.76       152

    accuracy                           0.83       430
   macro avg       0.81      0.81      0.81       430
weighted avg       0.83      0.83      0.83       430


Summary of all models for Dataset_3:
  model  accuracy  balanced_accuracy  f1_weighted  precision_weighted  \
0    rf  0.951163           0.942848     0.950972            0.951064   
1   xgb  0.946512           0.939251     0.946389            0.946366   
2   svc  0.855814           0.846744     0.856423            0.857380   
3    lr  0.811628           0.820026     0.815025            0.829169   
4   nas  0.830233           0.813541     0.830105            0.829987   
5    nb  0.783721           0.788006     0.787487            0.800458   
6   knn  0.781395           0.784717     0.78